In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pygame.midi

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Initialize MIDI
pygame.midi.init()
player = pygame.midi.Output(0)

# Define MIDI notes
finger_notes = {4: "C4", 8: "D4", 12: "E4", 16: "F4", 20: "G4"}
midi_notes = {"C4": 60, "D4": 62, "E4": 64, "F4": 65, "G4": 67}
active_notes = set()

# Start Video Capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = np.array([[lm.x, lm.y] for lm in hand_landmarks.landmark])

            for tip, note in finger_notes.items():
                if landmarks[tip][1] < landmarks[tip - 2][1]:  # Check if tip is above lower joint
                    if note not in active_notes:
                        player.note_on(midi_notes[note], 127)
                        active_notes.add(note)
                        print(f"Playing {note}")
                else:
                    if note in active_notes:
                        player.note_off(midi_notes[note], 127)
                        active_notes.remove(note)
                        print(f"Stopped {note}")

    cv2.imshow("Piano Hand Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
player.close()
pygame.midi.quit()
cv2.destroyAllWindows()